In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG

In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
tg.rsample(R_mu, sigma)

(tensor([[-0.0114, -0.4528, -0.4395,  0.7756]]),
 tensor([[[ 0.2035,  0.6922, -0.6924],
          [-0.6716,  0.6133,  0.4157],
          [ 0.7124,  0.3804,  0.5897]]]))

# Test torch_tsgaussian normal_term

In [7]:
sigma = torch.ones(3).reshape((1,3))
sigma

tensor([[1., 1., 1.]])

In [8]:
tg.normal_term(sigma)

tensor([15.7496])

# Test torch_tsgaussian log_map

In [9]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [10]:
tg.log_map(R_1, R_2)

tensor([0., 0., 0.])

# Test torch_tsgaussian log_probs

In [11]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [12]:
# tg.log_probs(R_x, R_mu, sigma)

In [13]:
np.e ** (-2.7568)

0.06349462641817973

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [14]:
tsg = TSG(None)

In [15]:
print(tsg.distribution)

In [16]:
tsg

In [17]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

(tensor([[ 0.3595,  0.7023, -0.4948,  0.3643]]),
 tensor([[[-0.4762,  0.8654,  0.1560],
          [ 0.1444,  0.2520, -0.9569],
          [-0.8674, -0.4331, -0.2450]]]))

# Test TangentSpaceGaussian log_prob_from_params

In [18]:
torch.eye(3).repeat(2,1,1).size()

torch.Size([2, 3, 3])

In [19]:
torch.ones(3).repeat(2,1).size()

torch.Size([2, 3])

In [20]:
# tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

In [21]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

tensor([[[ 3.8241e-01,  3.5724e-01,  2.5963e-02, -5.4199e-01],
         [ 1.3479e+00,  8.0512e-01,  2.6893e+00,  9.6555e-01],
         [-2.8228e-01,  2.2146e-01,  5.4266e-01,  4.6606e-01],
         [-9.6338e-01,  2.0056e-01, -1.0496e+00,  8.3331e-01]],

        [[-4.3615e-01, -3.4348e-04,  5.7952e-01, -1.4918e-01],
         [ 5.0859e-01,  1.2554e-01, -3.6044e-01, -2.1548e-01],
         [ 1.1555e+00,  4.1908e-01, -1.6460e-01,  3.1128e-01],
         [ 1.0291e+00,  6.6994e-01, -5.4879e-01,  2.7424e-01]]])

Again, codes can run, but need to check correctness.

# Try to run training

In [22]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [30]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 0.0,
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 110, eval_freq = 5, n_eval_episodes = 5)

In [31]:
from torch import autograd
with autograd.detect_anomaly():
    main(None)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.88    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 756      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.57    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 778      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.21    |
| time/              |          |
|    episodes        | 12       |
|    fps             |

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.07    |
| time/              |          |
|    episodes        | 100      |
|    fps             | 890      |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/tmp/ipykernel_5714/381243426.py:2: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():


vec:  torch.Size([1, 3]) torch.Size([1, 3, 3])
vec:  torch.Size([256, 3]) torch.Size([256, 3, 3])
actions:  tensor([[-0.4511,  0.0757,  0.5725,  0.6804],
        [-0.4546,  0.0672,  0.5619,  0.6879],
        [-0.4358,  0.0909,  0.5713,  0.6895],
        ...,
        [-0.4618,  0.0593,  0.5713,  0.6759],
        [-0.4563,  0.0674,  0.5680,  0.6816],
        [-0.4550,  0.0737,  0.5778,  0.6736]])
actions_mat:  tensor([[[ 0.3330, -0.8474, -0.4135],
         [ 0.7108, -0.0626,  0.7006],
         [-0.6196, -0.5272,  0.5815]],

        [[ 0.3596, -0.8340, -0.4184],
         [ 0.7119, -0.0447,  0.7009],
         [-0.6032, -0.5499,  0.5777]],

        [[ 0.3306, -0.8671, -0.3726],
         [ 0.7086, -0.0327,  0.7048],
         [-0.6233, -0.4970,  0.6037]],

        ...,

        [[ 0.3401, -0.8270, -0.4476],
         [ 0.7175, -0.0794,  0.6920],
         [-0.6079, -0.5565,  0.5664]],

        [[ 0.3457, -0.8359, -0.4264],
         [ 0.7128, -0.0616,  0.6987],
         [-0.6103, -0.5455,  0.574

/home/fantasticoven/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:145: UserWarning: Error detected in MseLossBackward. Traceback of forward call that caused the error:
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 707, in start
    self.io_loop.start()
  File "/home/fantasticoven/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py"

RuntimeError: Function 'MseLossBackward' returned nan values in its 0th output.

# Experiments for batch operations

In [25]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

tensor([[0.6065, 0.1806, 0.9259]])

In [26]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [27]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

ImportError: /home/fantasticoven/.local/lib/python3.8/site-packages/functorch/_C.so: undefined symbol: _ZNK3c104Type14isSubtypeOfExtERKS0_PSo

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)

# Question to ask: the original wahba problem action is (4,), in our case actions are (3,3).